In [2]:
%%pyspark
df = spark.read.load('abfss://files@datalakemwdvy04.dfs.core.windows.net/products/products.csv', format='csv'
# If header exists uncomment line below
, header=True
)
display(df.limit(10))

StatementMeta(sparkmwdvy04, 0, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c39a53c8-17df-44bf-9363-0aba62be1876)

In [3]:
 delta_table_path = "/delta/products-delta"
 df.write.format("delta").save(delta_table_path)

StatementMeta(sparkmwdvy04, 0, 4, Finished, Available, Finished)

In [4]:
 from delta.tables import *
 from pyspark.sql.functions import *

 # Create a deltaTable object
 deltaTable = DeltaTable.forPath(spark, delta_table_path)

 # Update the table (reduce price of product 771 by 10%)
 deltaTable.update(
     condition = "ProductID == 771",
     set = { "ListPrice": "ListPrice * 0.9" })

 # View the updated data as a dataframe
 deltaTable.toDF().show(10)

StatementMeta(sparkmwdvy04, 0, 5, Finished, Available, Finished)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes| 3059.991|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [5]:
 new_df = spark.read.format("delta").load(delta_table_path)
 new_df.show(10)

StatementMeta(sparkmwdvy04, 0, 6, Finished, Available, Finished)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes| 3059.991|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [6]:
display(new_df.limit(10))

StatementMeta(sparkmwdvy04, 0, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a0716cc7-6280-4648-b139-eb273f99403a)

In [7]:
 new_df = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
 new_df.show(10)

StatementMeta(sparkmwdvy04, 0, 8, Finished, Available, Finished)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [8]:
 deltaTable.history(10).show(20, False, True)

StatementMeta(sparkmwdvy04, 0, 9, Finished, Available, Finished)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 1                                                                                                                                                         
 timestamp           | 2024-08-25 05:23:48                                                                                                                                       
 userId              | null                                                                                                                                                      
 userName            | null                                                                                                                                                      
 operation           | UPDATE                                                                                 

In [9]:
 spark.sql("CREATE DATABASE AdventureWorks")
 spark.sql("CREATE TABLE AdventureWorks.ProductsExternal USING DELTA LOCATION '{0}'".format(delta_table_path))
 spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsExternal").show(truncate=False)

StatementMeta(sparkmwdvy04, 0, 10, Finished, Available, Finished)

+----------------------------+-----------------------------------------------------------------------+-------+
|col_name                    |data_type                                                              |comment|
+----------------------------+-----------------------------------------------------------------------+-------+
|ProductID                   |string                                                                 |       |
|ProductName                 |string                                                                 |       |
|Category                    |string                                                                 |       |
|ListPrice                   |string                                                                 |       |
|                            |                                                                       |       |
|# Partitioning              |                                                                       |       |
|

In [10]:
 %%sql

 USE AdventureWorks;

 SELECT * FROM ProductsExternal;

StatementMeta(sparkmwdvy04, 0, 12, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 295 rows and 4 fields>

In [11]:
 df.write.format("delta").saveAsTable("AdventureWorks.ProductsManaged")
 spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsManaged").show(truncate=False)

StatementMeta(sparkmwdvy04, 0, 13, Finished, Available, Finished)

+----------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                                                       |comment|
+----------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------+
|ProductID                   |string                                                                                                                          |       |
|ProductName                 |string                                                                                                                          |       |
|Category                    |string                                                                                                                          | 

In [12]:
 %%sql

 USE AdventureWorks;

 SELECT * FROM ProductsManaged;

StatementMeta(sparkmwdvy04, 0, 15, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 295 rows and 4 fields>

In [13]:
 %%sql

 USE AdventureWorks;

 SHOW TABLES;

StatementMeta(sparkmwdvy04, 0, 17, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 2 rows and 3 fields>

In [14]:
 %%sql

 USE AdventureWorks;

 DROP TABLE IF EXISTS ProductsExternal;
 DROP TABLE IF EXISTS ProductsManaged;

StatementMeta(sparkmwdvy04, 0, 20, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [15]:
 %%sql

 USE AdventureWorks;

 CREATE TABLE Products
 USING DELTA
 LOCATION '/delta/products-delta';

StatementMeta(sparkmwdvy04, 0, 22, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [16]:
 %%sql

 USE AdventureWorks;

 SELECT * FROM Products;

StatementMeta(sparkmwdvy04, 0, 24, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 295 rows and 4 fields>

In [17]:
 from notebookutils import mssparkutils
 from pyspark.sql.types import *
 from pyspark.sql.functions import *

 # Create a folder
 inputPath = '/data/'
 mssparkutils.fs.mkdirs(inputPath)

 # Create a stream that reads data from the folder, using a JSON schema
 jsonSchema = StructType([
 StructField("device", StringType(), False),
 StructField("status", StringType(), False)
 ])
 iotstream = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger", 1).json(inputPath)

 # Write some event data to the folder
 device_data = '''{"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev2","status":"error"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"error"}
 {"device":"Dev2","status":"ok"}
 {"device":"Dev2","status":"error"}
 {"device":"Dev1","status":"ok"}'''
 mssparkutils.fs.put(inputPath + "data.txt", device_data, True)
 print("Source stream created...")

StatementMeta(sparkmwdvy04, 0, 25, Finished, Available, Finished)

Source stream created...


In [18]:
 # Write the stream to a delta table
 delta_stream_table_path = '/delta/iotdevicedata'
 checkpointpath = '/delta/checkpoint'
 deltastream = iotstream.writeStream.format("delta").option("checkpointLocation", checkpointpath).start(delta_stream_table_path)
 print("Streaming to delta sink...")

StatementMeta(sparkmwdvy04, 0, 26, Finished, Available, Finished)

Streaming to delta sink...


In [19]:
 # Read the data in delta format into a dataframe
 df = spark.read.format("delta").load(delta_stream_table_path)
 display(df)

StatementMeta(sparkmwdvy04, 0, 27, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7c4cdfcc-f948-46a1-a873-3a555c1ee84f)

In [20]:
 # create a catalog table based on the streaming sink
 spark.sql("CREATE TABLE IotDeviceData USING DELTA LOCATION '{0}'".format(delta_stream_table_path))

StatementMeta(sparkmwdvy04, 0, 28, Finished, Available, Finished)

DataFrame[]

In [21]:
 %%sql

 SELECT * FROM IotDeviceData;

StatementMeta(sparkmwdvy04, 0, 29, Finished, Available, Finished)

<Spark SQL result set with 9 rows and 2 fields>

In [22]:
 # Add more data to the source stream
 more_data = '''{"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"error"}
 {"device":"Dev2","status":"error"}
 {"device":"Dev1","status":"ok"}'''

 mssparkutils.fs.put(inputPath + "more-data.txt", more_data, True)

StatementMeta(sparkmwdvy04, 0, 30, Finished, Available, Finished)

True

In [23]:
 %%sql

 SELECT * FROM IotDeviceData;

StatementMeta(sparkmwdvy04, 0, 31, Finished, Available, Finished)

<Spark SQL result set with 16 rows and 2 fields>

In [24]:
 deltastream.stop()

StatementMeta(sparkmwdvy04, 0, 32, Finished, Available, Finished)

In [25]:
 deltastream = iotstream.writeStream.format("delta").option("checkpointLocation", checkpointpath).start(delta_stream_table_path)
 print("Streaming to delta sink...")

StatementMeta(sparkmwdvy04, 0, 33, Finished, Available, Finished)

Streaming to delta sink...


In [26]:
 # Add more data to the source stream
 more_data2 = '''{"device":"Dev1","status":"ok"}
 {"device":"Dev3","status":"ok"}
 {"device":"Dev3","status":"ok"}'''

 mssparkutils.fs.put(inputPath + "more-data2.txt", more_data2, True)

StatementMeta(sparkmwdvy04, 0, 34, Finished, Available, Finished)

True